In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# dand wiki

In [159]:
# URL of the webpage containing the spell list
base_url = 'https://www.dandwiki.com'
spells_url = f'{base_url}/wiki/5e_SRD:Spells'

# Send a GET request to the webpage
response = requests.get(spells_url)
response.raise_for_status() 

In [160]:
soup = BeautifulSoup(response.content, 'html.parser')

In [173]:
def get_spell_details(spell_url):
    damage_types = ['piercing, bludgeoning, slashing, cold, fire, lightning, thunder, poison, acid, necrotic, radiant, force, psychic']
    response = requests.get(base_url + spell_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    spell_details = {}
    if soup.find('caption'):
        spell_name = soup.find('caption').text.strip()
        spell_details['Name'] = spell_name
    
        labels = soup.find('table', class_='d20').find('td').text.strip().split(' ')
        if '(ritual)' in labels:
            spell_details['Ritual'] = True
        else:
            spell_details['Ritual'] = False

        if 'cantrip' in labels:
            spell_details['Level'] = 'Cantrip'
            spell_details['School'] = labels[0]
        else:
            spell_details['Level'] = labels[0][0]
            spell_details['School'] = labels[1]
            
        spell_details_table = soup.find('table', class_='d20').find_all('tr')
        for row in spell_details_table:
            cells = row.find_all(['th', 'td'])
            if len(cells) == 2:
                key = cells[0].text.strip().rstrip(':')
                value = cells[1].text.strip()
                spell_details[key] = value
        description_paragraphs = soup.find_all('p')
        description = '\n'.join([p.text.strip() for p in description_paragraphs if not p.find('a', title="Main Page")])
    
    
        spell_details['Description'] = description
    
        damage_occurrences = len(re.findall('damage', description))
        init_index = 0
        damage_type_list = []
        for i in range(damage_occurrences):
            index_of_damage = description[init_index:].find('damage')
            if index_of_damage != -1:
                last_space_before_damage = description.rfind(" ", 0, index_of_damage-1)
                damage_type = description[last_space_before_damage+1:index_of_damage-1]
                if damage_type in damage_types:
                    damage_type_list.append(damage_type)
                init_index = index_of_damage
        if damage_type_list:
            spell_details['Damage Type'] = set(damage_type_list)
        else:
            spell_details['Damage Type'] = 'Non-damaging'
            
        index_of_attack = description.find('spell attack')
        if index_of_attack != -1:
            last_space_before_attack = description.rfind(" ", 0, index_of_attack-1)
            attack_type = description[last_space_before_attack+1:index_of_attack-1]
            spell_details['Attack Type'] = attack_type
        else:
            spell_details['Attack Type'] = None
        
        index_of_save = description.find('saving throw')
        if index_of_save != -1:
            last_space_before_save = description.rfind(" ", 0, index_of_save-1)
            save_type = description[last_space_before_save+1:index_of_save-1]
            spell_details['Saving Throw'] = save_type
        else:
            spell_details['Saving Throw'] = None
    
        
        return spell_details

In [175]:
spells_list = []
a_tag_list = []
ul_list = []
for ul in soup.find_all('ul')[5:15]: # find all unordered lists # Need 5 to 14 which are the spells
    ul_list.append (ul)
    for li in ul.find_all('li'): # find all list items per unordered list
        a_tag = li.find('a')
        a_tag_list.append(a_tag)
        if a_tag and 'href' in a_tag.attrs:
            spell_url = a_tag['href'] # this gets the relative url attached to a_tag
            spell_details = get_spell_details(spell_url)
            spells_list.append(spell_details)
            time.sleep(0.5) # be polite to the server

spells_list_filtered = [entry for entry in spells_list if entry is not None]
spells_df = pd.DataFrame(spells_list_filtered)

# Save the DataFrame to a CSV file
spells_df.to_csv('5e_spells.csv', index=False)
# spells_df = pd.DataFrame(spells_list)
#spells_df.to_csv('5e_spells.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'keys'

# Wikidot

* td - table data
* th - table header
* li - list
* tr - data row
* a - anchor tag - used to create hyperlinks
* p - defines paragraph
* strong - emphasis
* br - line break

In [190]:
def get_spell_details(base_and_spell_url):

    response = requests.get(base_and_spell_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    print(base_and_spell_url)
    spell_details = {}
    spell_info = soup.find_all('p')
    source = spell_info[0].text[8:]
    text = spell_info[1].text.split(' ')
    if 'cantrip' == text[1]:
        school = text[0]
        level = text[1]
    else:
        school = text[1]
        level = text[0]
    
    if 'ritual' in text:
        ritual = True
    else:
        ritual = False
    try:    
        time = spell_info[2].text.split('\n')[0][14:]
        range = spell_info[2].text.split('\n')[1][7:]
        comp = spell_info[2].text.split('\n')[2][12:]
        duration = spell_info[2].text.split('\n')[3][10:]
    except:
        text = ' '.join(p.text.strip() for p in spell_info)
        spell_name_level_match = re.match(r'^(.*?)\s+(cantrip|1st level|2nd level|3rd level|4th level|5th level|6th level|7th level|8th level|9th level)', text, re.IGNORECASE | re.MULTILINE)
        name = spell_name_level_match.group(1).strip() if spell_name_level_match else None
        level = spell_name_level_match.group(2).strip() if spell_name_level_match else None
    
        # Extracting Casting Time
        casting_time_match = re.search(r'Casting Time:\s*(.*)', text)
        duration = casting_time_match.group(1).strip() if casting_time_match else None
    
        # Extracting Range
        range_match = re.search(r'Range:\s*(.*)', text)
        range = range_match.group(1).strip() if range_match else None
    
        # Extracting Components
        components_match = re.search(r'Components:\s*(.*)', text)
        comp = [c.strip() for c in components_match.group(1).split(',')] if components_match else None
    
        # Extracting Duration
        duration_match = re.search(r'Duration:\s*(.*)', text)
        spell_details['duration'] = duration_match.group(1).strip() if duration_match else None
    description = ' '.join(p.text.strip() for p in spell_info[3:]).lower()
    pattern = r'\bspell lists[.: -]'
    match = re.search(pattern, description)
    # if not match:
    #     pattern = r'\bSpell Lists\:'
    #     match = re.search(pattern, description)
    
    spell_lists = description[match.end()+1:]
    description = description[:match.start()-1]
        # else:
        #     spell_lists = spell_info[5].text[13:]
    # pattern = r'\d+d\d+' for dice
    # Find all occurrences of the pattern in the paragraph
    # dice_notations = re.findall(pattern, paragraph)
    save = False
    attack = False
    save_type = False
    attack_type = False
    if 'saving throw' in description:
        save = True
        indices = [index-1 for index, value in enumerate(description.split(' ')) if value == 'saving']
        save_type = [description.split(' ')[index] for index in indices]
    if 'spell attack' in description:
        attack = True
        pattern = r'\b(melee|ranged)\s+spell\s+attack\b'
        match = re.search(pattern, description, re.IGNORECASE)
        if match:
            attack_type = match.group()

    spell_details['name'] = base_and_spell_url[31:].replace('-',' ').title()
    spell_details['source'] = source
    spell_details['school'] = school
    spell_details['level'] = level
    spell_details['ritual'] = ritual
    spell_details['time'] = time
    spell_details['range'] = range
    spell_details['components'] = comp
    spell_details['duration'] = duration
    spell_details['description'] = description
    spell_details['save'] = save
    spell_details['save_type'] = save_type
    spell_details['attack'] = attack
    spell_details['attack_type'] = attack_type
    spell_details['Lists'] = spell_lists
    
    return spell_details
    
    

In [191]:
# URL of the webpage containing the spell list
base_url = 'http://dnd5e.wikidot.com'
spells_url = f'{base_url}/spells'

# Send a GET request to the webpage
response = requests.get(spells_url)
response.raise_for_status() 

In [192]:
soup = BeautifulSoup(response.content, 'html.parser')

In [194]:
spell_levels = ['Cantrip', 1,2,3,4,5,6,7,8,9]
ct = 0
spells_list = []
dud_list = []
for level_table in soup.find_all('table'):
    spell_level = spell_levels[ct]
    for a_tag in level_table.find_all('a'):
        if a_tag and 'href' in a_tag.attrs:
            spell_url = a_tag['href']
            spell_details = get_spell_details(base_url+spell_url)
            spells_list.append(spell_details)
            time.sleep(0.5) # be polite to the server
spells_df = pd.DataFrame(spells_list)

http://dnd5e.wikidot.com/spell:acid-splash
http://dnd5e.wikidot.com/spell:blade-ward
http://dnd5e.wikidot.com/spell:booming-blade
http://dnd5e.wikidot.com/spell:chill-touch
http://dnd5e.wikidot.com/spell:control-flames
http://dnd5e.wikidot.com/spell:create-bonfire
http://dnd5e.wikidot.com/spell:dancing-lights
http://dnd5e.wikidot.com/spell:decompose
http://dnd5e.wikidot.com/spell:druidcraft
http://dnd5e.wikidot.com/spell:eldritch-blast
http://dnd5e.wikidot.com/spell:encode-thoughts
http://dnd5e.wikidot.com/spell:fire-bolt
http://dnd5e.wikidot.com/spell:friends
http://dnd5e.wikidot.com/spell:frostbite
http://dnd5e.wikidot.com/spell:green-flame-blade
http://dnd5e.wikidot.com/spell:guidance
http://dnd5e.wikidot.com/spell:gust
http://dnd5e.wikidot.com/spell:hand-of-radiance
http://dnd5e.wikidot.com/spell:infestation
http://dnd5e.wikidot.com/spell:light
http://dnd5e.wikidot.com/spell:lightning-lure
http://dnd5e.wikidot.com/spell:mage-hand
http://dnd5e.wikidot.com/spell:magic-stone
http://dn

In [196]:
spells_df.to_csv('5e_spells Wikidot.csv', index=False)

In [195]:
spells_df

,name,source,school,level,ritual,time,range,components,duration,description,save,save_type,attack,attack_type,Lists
0,Acid Splash,Player's Handbook,Conjuration,cantrip,False,1 action,60 feet,"V, S",Instantaneous,you hurl a bubble of acid. choose one creature...,True,[dexterity],False,False,"artificer, sorcerer, wizard"
1,Blade Ward,Player's Handbook,Abjuration,cantrip,False,1 action,Self,"V, S",1 round,you extend your hand and trace a sigil of ward...,False,False,False,False,"bard, sorcerer, warlock, wizard"
2,Booming Blade,Tasha's Cauldron of Everything,Evocation,cantrip,False,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",1 round,you brandish the weapon used in the spell’s ca...,False,False,False,False,"artificer, sorcerer (optional), warlock (optio..."
3,Chill Touch,Player's Handbook,Necromancy,cantrip,False,1 action,120 feet,"V, S",1 round,"you create a ghostly, skeletal hand in the spa...",False,False,True,ranged spell attack,"sorcerer, warlock, wizard"
4,Control Flames,Xanathar's Guide to Everything,Transmutation,cantrip,False,1 action,60 feet,S,Instantaneous or 1 hour,you choose nonmagical flame that you can see w...,False,False,False,False,"druid, sorcerer, wizard"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,Time Stop,Player's Handbook,transmutation,9th-level,False,1 action,Self,V,Instantaneous,you briefly stop the flow of time for everyone...,False,False,False,False,"sorcerer, wizard"
576,True Polymorph,Player's Handbook,transmutation,9th-level,False,1 action,30 feet,"V, S, M (a drop of mercury, a dollop of gum ar...","Concentration, up to 1 hour",choose one creature or nonmagical object that ...,True,[wisdom],False,False,"bard, warlock, wizard"
577,True Resurrection,Player's Handbook,necromancy,9th-level,False,1 hour,Touch,"V, S, M (a sprinkle of holy water and diamonds...",Instantaneous,you touch a creature that has been dead for no...,False,False,False,False,"cleric, druid"
578,Weird,Player's Handbook,illusion,9th-level,False,1 action,120 feet,"V, S","Concentration, up to 1 minute",drawing on the deepest fears of a group of cre...,True,"[wisdom, wisdom]",False,False,"warlock (optional), wizard"
